In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Home Energy Footprint Optimizer 🌱

**Track:** Agents for Good – Sustainability  
**Goal:** Analyze a simple CSV of home appliance usage, identify the biggest “energy hogs”, and generate a human-readable action plan with estimated savings.

This notebook demonstrates:

- A **multi-agent system** using Gemini + ADK:
  - Data Ingestion Agent
  - Optimizer Agent
  - Report Agent
  - (Optional) Judge Agent for evaluation
- **Custom tools** for CSV loading and energy calculations
- **Observability** via ADK's `LoggingPlugin`
- A small **sessions demo** using `InMemorySessionService` to illustrate stateful agents


In [2]:
!pip install -q google-adk pandas

import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ GOOGLE_API_KEY loaded from Kaggle secrets")
except Exception as e:
    print("❌ Could not load GOOGLE_API_KEY. Please add it to Kaggle secrets.")
    raise e


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 11.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.10.0 w

In [3]:
import json
from typing import List, Dict, Any, Optional

import pandas as pd

from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner, Runner
from google.adk.sessions import InMemorySessionService
from google.adk.plugins.logging_plugin import LoggingPlugin
# AgentTool is available if you later want agents-as-tools
from google.adk.tools.agent_tool import AgentTool

retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

print("✅ ADK imports and retry config ready")

✅ ADK imports and retry config ready


In [4]:
def load_energy_data(csv_path: str) -> str:
    """
    Tool: Load a CSV of appliance usage and return it as JSON (list of records).

    Expected columns:
      - appliance (str)
      - wattage (number, watts)
      - hours_per_day (number)
      - days_per_month (number)
      - cost_per_kwh (number, e.g. 0.15)
    """
    df = pd.read_csv(csv_path)
    df = df.fillna(0)
    return df.to_json(orient="records")


def compute_energy_stats(data_json: str) -> str:
    """
    Tool: Given JSON records with wattage/hours/etc, compute kWh and cost per month.

    Adds:
      - kwh_per_month
      - cost_per_month
    """
    records: List[Dict[str, Any]] = json.loads(data_json)

    for rec in records:
        wattage = float(rec.get("wattage", 0) or 0)
        hours_per_day = float(rec.get("hours_per_day", 0) or 0)
        days_per_month = float(rec.get("days_per_month", 0) or 0)
        cost_per_kwh = float(rec.get("cost_per_kwh", 0) or 0)

        kwh_per_month = wattage * hours_per_day * days_per_month / 1000.0
        cost_per_month = kwh_per_month * cost_per_kwh

        rec["kwh_per_month"] = round(kwh_per_month, 2)
        rec["cost_per_month"] = round(cost_per_month, 2)

    return json.dumps(records, indent=2)

def event_to_text(response):
    """
    Convert ADK runner.run_debug(...) output into plain text.
    Handles:
      - str
      - list of Event objects (InMemoryRunner output)
    """
    # If it's already a plain string, just return it
    if isinstance(response, str):
        return response

    # If it's a list (typical for run_debug)
    if isinstance(response, list) and len(response) > 0:
        last = response[-1]
        try:
            content = getattr(last, "content", None)
            if content is not None and getattr(content, "parts", None):
                texts = []
                for part in content.parts:
                    # Only include parts that have text
                    if hasattr(part, "text") and part.text is not None:
                        texts.append(part.text)
                if texts:
                    return "\n\n".join(texts)
        except Exception:
            # Fallback: just string-ify the last event
            return str(last)

    # Fallback: string-ify whatever it is
    return str(response)

print("✅ Tools defined: load_energy_data, compute_energy_stats, event_to_text")


✅ Tools defined: load_energy_data, compute_energy_stats, event_to_text


In [5]:
data_ingestion_agent = LlmAgent(
    name="data_ingestion_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Loads and normalizes home energy usage data.",
    instruction=(
        "You are responsible for preparing home energy usage data so that other agents "
        "can analyze it.\n\n"
        "Given a CSV path, you MUST:\n"
        "1) Call the `load_energy_data` tool to read raw rows.\n"
        "2) Call the `compute_energy_stats` tool to add kwh_per_month and cost_per_month "
        "   for each appliance.\n"
        "3) Return ONLY the final JSON list of records with these fields:\n"
        "   appliance, wattage, hours_per_day, days_per_month, cost_per_kwh, "
        "   kwh_per_month, cost_per_month.\n"
    ),
    tools=[load_energy_data, compute_energy_stats],
)

print("✅ Data Ingestion Agent created")


✅ Data Ingestion Agent created


In [6]:
optimizer_agent = LlmAgent(
    name="optimizer_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Analyzes normalized energy data and suggests optimizations.",
    instruction=(
        "You receive a JSON list of appliances with kwh_per_month and cost_per_month.\n\n"
        "Your tasks:\n"
        "1) Parse the JSON.\n"
        "2) Compute total_kwh and total_cost across all appliances.\n"
        "3) Rank appliances by cost_per_month (highest first).\n"
        "4) Identify the top contributors (e.g. top 3–5 devices by cost).\n"
        "5) For each top device, propose specific, realistic behavior changes. Examples:\n"
        "   - reduce hours_per_day\n"
        "   - shift usage to off-peak hours\n"
        "   - consider upgrading to a more efficient model (only when reasonable)\n"
        "6) Estimate potential monthly savings per device AND overall.\n\n"
        "Return a structured JSON object with keys:\n"
        "  - total_kwh\n"
        "  - total_cost\n"
        "  - top_appliances: list of objects with fields\n"
        "      appliance, cost_per_month, kwh_per_month, percentage_of_total,\n"
        "      suggestions, estimated_savings\n"
        "  - total_estimated_savings\n"
    ),
)

print("✅ Optimizer Agent created")


✅ Optimizer Agent created


In [7]:
report_agent = LlmAgent(
    name="report_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Generates a human-friendly summary report.",
    instruction=(
        "You receive a JSON object describing the energy analysis, with fields like:\n"
        "  total_kwh, total_cost, top_appliances, total_estimated_savings.\n\n"
        "Write a concise, friendly report for a non-technical user. The report should:\n"
        "1) Briefly summarize total energy and cost.\n"
        "2) List the top 3–5 appliances driving energy usage, with percentages.\n"
        "3) For each, explain what the user can do differently (in simple language).\n"
        "4) Highlight the estimated monthly savings if they follow those changes.\n\n"
        "Use short paragraphs and bullet points where helpful. Do NOT return JSON. "
        "Return plain text suitable for showing directly to the user."
    ),
)

print("✅ Report Agent created")


✅ Report Agent created


In [8]:
judge_agent = LlmAgent(
    name="judge_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Evaluates the quality of the generated energy-saving recommendations.",
    instruction=(
        "You are an evaluation agent. You will receive:\n"
        "1) The normalized energy usage data (JSON).\n"
        "2) The optimizer's structured analysis (JSON).\n"
        "3) The final human-readable report text.\n\n"
        "Your task is to rate, on a scale from 1 to 10, how:\n"
        "- actionable\n"
        "- consistent with the data\n"
        "- clear and understandable\n"
        "the report is.\n\n"
        "Return a short JSON object like:\n"
        "{ 'actionability': <1-10>, 'consistency': <1-10>, 'clarity': <1-10>, 'comments': '...'}\n"
        "No extra explanation beyond this JSON object."
    ),
)

print("✅ Judge Agent created")


✅ Judge Agent created


In [9]:
async def run_energy_analysis(csv_path: str, evaluate: bool = False) -> Dict[str, Any]:
    """
    Orchestrates the full flow:
      1. Data ingestion & normalization
      2. Optimization & suggestions
      3. Human-friendly report generation
      4. (Optional) Evaluation via judge agent
    """

    # --- Step 1: Data ingestion with observability ---
    ingestion_runner = InMemoryRunner(
        agent=data_ingestion_agent,
        plugins=[LoggingPlugin()],
    )

    ingestion_prompt = (
        "Load and normalize the home energy usage data from this CSV path:\n"
        f"{csv_path}\n\n"
        "Use your tools to:\n"
        "1) load the rows\n"
        "2) compute kwh_per_month and cost_per_month\n"
        "Return ONLY the final JSON list of records."
    )
    normalized_data_raw = await ingestion_runner.run_debug(ingestion_prompt)
    normalized_data = event_to_text(normalized_data_raw)

    # --- Step 2: Optimization with observability ---
    optimizer_runner = InMemoryRunner(
        agent=optimizer_agent,
        plugins=[LoggingPlugin()],
    )

    optimization_prompt = (
        "You are the Optimizer Agent. Here is the normalized energy dataset as JSON.\n"
        "Analyze it and produce the structured analysis as requested in your instructions.\n\n"
        f"{normalized_data}"
    )
    analysis_raw = await optimizer_runner.run_debug(optimization_prompt)
    analysis = event_to_text(analysis_raw)

    # --- Step 3: Report generation with observability ---
    report_runner = InMemoryRunner(
        agent=report_agent,
        plugins=[LoggingPlugin()],
    )

    report_prompt = (
        "You are the Report Agent. Create a user-facing report based on this analysis JSON:\n\n"
        f"{analysis}"
    )
    report_raw = await report_runner.run_debug(report_prompt)
    report_text = event_to_text(report_raw)

    # --- Step 4: Optional evaluation ---
    evaluation_result: Optional[str] = None
    if evaluate:
        judge_runner = InMemoryRunner(
            agent=judge_agent,
            plugins=[LoggingPlugin()],
        )

        judge_prompt = (
            "You are the Judge Agent. Use the instructions you were given.\n\n"
            "Normalized data JSON:\n"
            f"{normalized_data}\n\n"
            "Analysis JSON:\n"
            f"{analysis}\n\n"
            "Human-friendly report text:\n"
            f"{report_text}\n\n"
            "Return only the JSON evaluation object."
        )
        eval_raw = await judge_runner.run_debug(judge_prompt)
        evaluation_result = event_to_text(eval_raw)

    return {
        "normalized_data": normalized_data,
        "analysis": analysis,
        "report_text": report_text,
        "evaluation": evaluation_result,
    }

print("✅ Orchestrator defined with LoggingPlugin for observability")


✅ Orchestrator defined with LoggingPlugin for observability


In [10]:
sample_csv_path = "energy_usage_sample.csv"

sample_df = pd.DataFrame([
    {"appliance": "Fridge",          "wattage": 150,  "hours_per_day": 24, "days_per_month": 30, "cost_per_kwh": 0.15},
    {"appliance": "Air Conditioner", "wattage": 1200, "hours_per_day": 6,  "days_per_month": 25, "cost_per_kwh": 0.15},
    {"appliance": "Washing Machine","wattage": 500,  "hours_per_day": 1,  "days_per_month": 12, "cost_per_kwh": 0.15},
    {"appliance": "TV",              "wattage": 100,  "hours_per_day": 4,  "days_per_month": 25, "cost_per_kwh": 0.15},
    {"appliance": "Laptop",          "wattage": 60,   "hours_per_day": 6,  "days_per_month": 25, "cost_per_kwh": 0.15},
])

sample_df.to_csv(sample_csv_path, index=False)
sample_df


,appliance,wattage,hours_per_day,days_per_month,cost_per_kwh
0,Fridge,150,24,30,0.15
1,Air Conditioner,1200,6,25,0.15
2,Washing Machine,500,1,12,0.15
3,TV,100,4,25,0.15
4,Laptop,60,6,25,0.15


In [11]:
result = await run_energy_analysis(sample_csv_path, evaluate=True)
print("=== Normalized Data (JSON) ===")
print(result["normalized_data"])

print("\n\n=== Analysis (JSON) ===")
print(result["analysis"])

print("\n\n=== Final User-Facing Report ===")
print(result["report_text"])

print("\n\n=== Evaluation (Judge Agent) ===")
print(result["evaluation"])



 ### Created new session: debug_session_id

User > Load and normalize the home energy usage data from this CSV path:
energy_usage_sample.csv

Use your tools to:
1) load the rows
2) compute kwh_per_month and cost_per_month
Return ONLY the final JSON list of records.
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-afc3710a-0e0b-42ab-b59c-9b0d96b63d77
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: InMemoryRunner
[logging_plugin]    Root Agent: data_ingestion_agent
[logging_plugin]    User Content: text: 'Load and normalize the home energy usage data from this CSV path:
energy_usage_sample.csv

Use your tools to:
1) load the rows
2) compute kwh_per_month and cost_per_month
Return ONLY the final JSON li...'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-afc3710a-0e0b-42ab-b59c-9b0d96b63d77
[logging_plugin]    Starting Agent: data_ingestion_agent
[logging_plu

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: data_ingestion_agent
[logging_plugin]    Content: function_call: load_energy_data
[logging_plugin]    Token Usage - Input: 432, Output: 25
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 487be3ac-5abb-420c-8175-3b677f83159d
[logging_plugin]    Author: data_ingestion_agent
[logging_plugin]    Content: function_call: load_energy_data
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['load_energy_data']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: load_energy_data
[logging_plugin]    Agent: data_ingestion_agent
[logging_plugin]    Function Call ID: adk-49913c4e-a0d3-4703-9857-24a41b4035b9
[logging_plugin]    Arguments: {'csv_path': 'energy_usage_sample.csv'}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: load_energy_data
[logging_plugin]    Agent: data_ingestion_agent
[logging_plugin]    Function Call ID: adk-49913c4e-a0d3-4703-9857-24a41b4035b9
[logg

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: data_ingestion_agent
[logging_plugin]    Content: function_call: compute_energy_stats
[logging_plugin]    Token Usage - Input: 687, Output: 232
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 97030aa1-6727-4359-964f-588d73b385dd
[logging_plugin]    Author: data_ingestion_agent
[logging_plugin]    Content: function_call: compute_energy_stats
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['compute_energy_stats']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: compute_energy_stats
[logging_plugin]    Agent: data_ingestion_agent
[logging_plugin]    Function Call ID: adk-1f3bcd60-e7db-4a3b-a52d-4286615ebf89
[logging_plugin]    Arguments: {'data_json': '[{"appliance":"Fridge","wattage":150,"hours_per_day":24,"days_per_month":30,"cost_per_kwh":0.15},{"appliance":"Air Conditioner","wattage":1200,"hours_per_day":6,"days_per_month":25,"cost_per_kwh":0.15},{"appliance":"Washin

In [12]:
from IPython.display import Markdown, display

display(Markdown("## Final Energy-Saving Recommendations"))
display(Markdown(result["report_text"]))


## Final Energy-Saving Recommendations

Here's a quick summary of your energy usage and how you can save!

**Your Energy Snapshot**

This month, your home used 313 kWh of energy, costing you approximately $47.15.

**Where Your Energy Goes**

The biggest energy users in your home are:

*   **Air Conditioner (57.3%)**: This is your largest energy consumer.
*   **Fridge (34.4%)**: Your refrigerator also uses a significant portion of your energy.
*   **TV (3.2%)**: Your television is a moderate energy user.
*   **Washing Machine (1.9%)**: This is a smaller contributor to your overall usage.

**Tips to Save Energy and Money**

Here are some simple changes you can make for the biggest impact:

*   **Air Conditioner**:
    *   Try setting your thermostat a degree or two cooler when you're away or it's cooler outside.
    *   Make sure your home is well-insulated and seal up any drafts around windows and doors.
    *   Regularly clean or replace your air filters.
*   **Fridge**:
    *   Check that the door seals are tight so cold air doesn't escape.
    *   Avoid putting hot food straight into the fridge – let it cool a bit first.
    *   Keep your fridge from being too full, allowing air to circulate better.
*   **TV**:
    *   Remember to turn it off when nobody is watching for a while.
    *   Lowering the screen brightness can also help.
*   **Washing Machine**:
    *   Wash your clothes using cold water whenever possible.
    *   Try to run full loads to get the most out of each cycle.

**Potential Monthly Savings**

By making these small adjustments, you could save around **$7.26** per month on your energy bill!

In [13]:
# Simple "Energy Coach" agent to demonstrate sessions

energy_coach_agent = LlmAgent(
    name="energy_coach_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="A simple conversational energy coach that remembers the user's home type.",
    instruction=(
        "You are an energy coach. Ask the user about their home (apartment, house, etc.) "
        "and remember it in the conversation. Use that context in future replies."
    ),
)

session_service = InMemorySessionService()

coach_runner = Runner(
    agent=energy_coach_agent,
    session_service=session_service,
    app_name="energy_coach_app",
)


async def demo_energy_coach_sessions():
    print("First turn (introduce home):")
    r1 = await coach_runner.run_debug(
        "Hi, I live in a small apartment with one AC and a fridge.",
        user_id="user123",
        session_id="session1",
    )
    print(r1)

    print("\nSecond turn (coach should remember home type):")
    r2 = await coach_runner.run_debug(
        "What are your top 2 tips for reducing my energy usage?",
        user_id="user123",
        session_id="session1",
    )
    print(r2)

print("✅ Sessions demo setup complete. Run `asyncio.run(demo_energy_coach_sessions())` to see it.")


✅ Sessions demo setup complete. Run `asyncio.run(demo_energy_coach_sessions())` to see it.


In [14]:
await demo_energy_coach_sessions()


First turn (introduce home):

 ### Created new session: session1

User > Hi, I live in a small apartment with one AC and a fridge.
energy_coach_agent > Hi there! Thanks for reaching out. It's great you're thinking about your energy use in your small apartment. Knowing you have one AC and a fridge helps me tailor my advice to you.

To start, could you tell me a little more about how you typically use your AC? For example:

*   What temperature do you usually set it to?
*   Do you run it all day, or just at certain times?

This will give me a good starting point!
[Event(model_version='gemini-2.5-flash-lite', content=Content(
  parts=[
    Part(
      text="""Hi there! Thanks for reaching out. It's great you're thinking about your energy use in your small apartment. Knowing you have one AC and a fridge helps me tailor my advice to you.

To start, could you tell me a little more about how you typically use your AC? For example:

*   What temperature do you usually set it to?
*   Do you run